<a href="https://colab.research.google.com/github/mleyvaz/computacion-paralela/blob/main/CudaNumpy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
from numba import cuda
import numpy
print(cuda.gpus)
import math

<Managed Device 0>


In [9]:
@cuda.jit
def matmul(A, B, C):
    """Perform matrix multiplication of C = A * B
    """
    row, col = cuda.grid(2)
    if row < C.shape[0] and col < C.shape[1]:
        tmp = 0.
        for k in range(A.shape[1]):
            tmp += A[row, k] * B[k, col]
        C[row, col] = tmp

In [12]:
# Initialize the data arrays
A = numpy.full((24, 12), 3, numpy.float) # matrix containing all 3's
B = numpy.full((12, 22), 4, numpy.float) # matrix containing all 4's

# Copy the arrays to the device
A_global_mem = cuda.to_device(A)
B_global_mem = cuda.to_device(B)

# Allocate memory on the device for the result
C_global_mem = cuda.device_array((24, 22))

# Configure the blocks
threadsperblock = (16, 16)
blockspergrid_x = int(math.ceil(A.shape[0] / threadsperblock[0]))
blockspergrid_y = int(math.ceil(B.shape[1] / threadsperblock[1]))
blockspergrid = (blockspergrid_x, blockspergrid_y)

# Start the kernel 
matmul[blockspergrid, threadsperblock](A_global_mem, B_global_mem, C_global_mem)

# Copy the result back to the host
C = C_global_mem.copy_to_host()

print(C)

[[144. 144. 144. 144. 144. 144. 144. 144. 144. 144. 144. 144. 144. 144.
  144. 144. 144. 144. 144. 144. 144. 144.]
 [144. 144. 144. 144. 144. 144. 144. 144. 144. 144. 144. 144. 144. 144.
  144. 144. 144. 144. 144. 144. 144. 144.]
 [144. 144. 144. 144. 144. 144. 144. 144. 144. 144. 144. 144. 144. 144.
  144. 144. 144. 144. 144. 144. 144. 144.]
 [144. 144. 144. 144. 144. 144. 144. 144. 144. 144. 144. 144. 144. 144.
  144. 144. 144. 144. 144. 144. 144. 144.]
 [144. 144. 144. 144. 144. 144. 144. 144. 144. 144. 144. 144. 144. 144.
  144. 144. 144. 144. 144. 144. 144. 144.]
 [144. 144. 144. 144. 144. 144. 144. 144. 144. 144. 144. 144. 144. 144.
  144. 144. 144. 144. 144. 144. 144. 144.]
 [144. 144. 144. 144. 144. 144. 144. 144. 144. 144. 144. 144. 144. 144.
  144. 144. 144. 144. 144. 144. 144. 144.]
 [144. 144. 144. 144. 144. 144. 144. 144. 144. 144. 144. 144. 144. 144.
  144. 144. 144. 144. 144. 144. 144. 144.]
 [144. 144. 144. 144. 144. 144. 144. 144. 144. 144. 144. 144. 144. 144.
  144. 1